In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.signal import find_peaks
import biosppy
import h5pickle as h5py
from tqdm import tqdm

In [ ]:
train_hdf = h5py.File('./input/train_dccweek2023.h5', 'r')
test_hdf = h5py.File('./input/test_dccweek2023.h5', 'r')

train_labels = pd.read_csv('./input/train_dccweek2023-labels.csv')
train_labels.columns = ['ids', 'classes']
ecg_types = ['DI', 'DII', 'DIII', 'AVR', 'AVL', 'AVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
ecg_types_dict = {'DI': 0, 'DII': 1, 'DIII': 2, 'AVR': 3, 'AVL': 4, 'AVF': 5, 'V1': 6, 'V2': 7, 'V3': 8, 'V4': 9, 'V5': 10, 'V6': 11}

In [ ]:
def features_stats(x, preffix=''):
    stats = {}
    stats['avg'] = np.mean(x)
    stats['std'] = np.std(x)
    stats['var'] = np.var(x)
    stats['min'] = np.min(x)
    stats['max'] = np.max(x)
    stats['q25'] = np.quantile(x, .25)
    stats['q50'] = np.quantile(x, .50)
    stats['q75'] = np.quantile(x, .75)
    # stats['count'] = len(x)
    # stats['nunique'] = len(np.unique(x))
    # stats['kurt'] = kurtosis(x)
    # stats['skew'] = skew(x)
    stats = {preffix+k: v for k, v in stats.items()} # Add preffix
    return stats

In [ ]:
def process_hdf(hdf):
    train_ids = np.array(hdf['exam_id'])
    tracings = hdf['tracings']
    features_list = []
    #for id in tqdm(range(len(train_ids))):
    for id in tqdm(range(100)):
        features = {'exam_id': train_ids[id]}
        for ecg in range(len(ecg_types)):
            values = np.array(tracings[id, :, ecg])
            try:
                ts, filtered, rpeaks, template_ts, templates, hear_rate_ts, heart_rate = biosppy.signals.ecg.ecg(values, sampling_rate=400, show=False)
                diff_templates = np.diff(templates)
                diff_heart_rate = np.diff(heart_rate)
                raw_to_filtered_diff = values - filtered
                features.update(features_stats(raw_to_filtered_diff, preffix=f'{ecg_types[ecg]}_raw_diff_'))
                features.update(features_stats(filtered, preffix=f'{ecg_types[ecg]}_'))
                features.update(features_stats(filtered[rpeaks], preffix=f'{ecg_types[ecg]}_peaks_'))
                features.update(features_stats(heart_rate, preffix=f'{ecg_types[ecg]}_heart_rate_'))
                features.update(features_stats(diff_heart_rate, preffix=f'{ecg_types[ecg]}_heart_rate_diff'))
                features.update(features_stats(templates.mean(axis=0), preffix=f'{ecg_types[ecg]}_templates_'))
                features.update(features_stats(templates.std(axis=0), preffix=f'{ecg_types[ecg]}_templates_std_'))
                features.update(features_stats(diff_templates.mean(axis=0), preffix=f'{ecg_types[ecg]}_templates_diff_'))
                features.update(features_stats(diff_templates.std(axis=0), preffix=f'{ecg_types[ecg]}_templates_std_diff_'))
            except Exception as e:
                #print(e)
                pass    
        features_list.append(features)        
    return pd.DataFrame(features_list)

In [ ]:
train_df = process_hdf(train_hdf)

In [ ]:
test_df = process_hdf(test_hdf)

In [ ]:
train_df.to_csv('./input/v1processed_train.csv', index=False)
test_df.to_csv('./input/v1processed_test.csv', index=False)

In [ ]:
def find_peaks(template):    
    r_peak = np.argmax(template)    
    q_peak = np.argmin(template[:r_peak])    
    s_peak = np.argmin(template[r_peak:]) + len(template[:r_peak])    
    p_peak = np.argmax(template[:q_peak])    
    t_peak = np.argmax(template[s_peak:]) + len(template[:s_peak])
    return r_peak, q_peak, s_peak, p_peak, t_peak

In [ ]:
import pywt
import scipy

def frequency_domain_features(signal, fs=400, low_freq_band=[4, 15], high_freq_band=[15, 40], very_low_freq_band=[0, 4]):
    # Calculate power spectral density
    freqs, psd = scipy.signal.welch(signal, fs)
    # Calculate spectral power in different frequency bands
    lf_power = np.trapz(psd[(freqs >= low_freq_band[0]) & (freqs <= low_freq_band[1])])
    hf_power = np.trapz(psd[(freqs >= high_freq_band[0]) & (freqs <= high_freq_band[1])])
    vlf_power = np.trapz(psd[(freqs >= very_low_freq_band[0]) & (freqs <= very_low_freq_band[1])])
    # Calculate spectral centroid, bandwidth, and entropy
    spectral_centroid = np.sum(freqs * psd) / np.sum(psd)
    spectral_bandwidth = np.sqrt(np.sum(psd * (freqs - spectral_centroid)**2) / np.sum(psd))
    spectral_entropy = -np.sum(psd * np.log2(psd))
    # Calculate wavelet transform coefficients
    coeffs = pywt.wavedec(signal, wavelet='db4', level=6)
    # Extract wavelet transform features
    cA6, cD6, cD5, cD4, cD3, cD2, cD1 = coeffs
    energy = [np.sum(np.square(cA6)), np.sum(np.square(cD6)), np.sum(np.square(cD5)),
              np.sum(np.square(cD4)), np.sum(np.square(cD3)), np.sum(np.square(cD2)), np.sum(np.square(cD1))]
    std = [np.std(cA6), np.std(cD6), np.std(cD5), np.std(cD4), np.std(cD3), np.std(cD2), np.std(cD1)]
    return lf_power, hf_power, vlf_power, spectral_centroid, spectral_bandwidth, spectral_entropy, energy, std

In [ ]:
def process_features(hdf, sample=None):
    # Open file
    f = hdf
    traces_ids = np.array(f['exam_id'])
    X = f['tracings']
    # Get Dimensions
    num_samples = X.shape[0] if sample is None else sample
    num_leads = X.shape[2]
    
    features_list = []
    for i in tqdm(range(num_samples)):        
        ecg_data = X[i, :, :]
        features = {'exam_id': traces_ids[i]}
        for j in range(num_leads):
            # Filter signal padding
            ecg_signal = np.trim_zeros(ecg_data[:, j])
            try:
                ts, filtered, rpeaks, template_ts, templates, hear_rate_ts, heart_rate = biosppy.signals.ecg.ecg(
                    ecg_signal, sampling_rate=400, show=False
                )
            except:
                pass      
            ######################################## Time-Domain Features ########################################            
            # Calculate time intervals between consecutive R-peaks (in samples)
            rr_intervals = np.diff(rpeaks)            
            # Convert time intervals to seconds
            rr_intervals_sec = rr_intervals / 400.0
            # Compute heart rate (in BPM)
            hr = 60.0 / rr_intervals_sec
            # Compute average heart rate over a specific period (e.g., 1 minute)
            average_hr = np.mean(hr)
            features[f'lead{j}_average_hr'] = average_hr
            # Average RR-Interval
            features[f'lead{j}_average_rr_interval'] = np.mean(rr_intervals)
            # Other intervals
            pr_intervals = []
            qrs_durations = []
            qt_intervals = []
            for template in templates:
                if template.max() < np.abs(template.min()):
                    template = -template
                try:
                    r_peak, q_peak, s_peak, p_peak, t_peak = find_peaks(template)
                    pr_intervals.append(q_peak - p_peak)
                    qrs_durations.append(s_peak - q_peak)
                    qt_intervals.append(t_peak - q_peak)
                except:
                    pass
            features[f'lead{j}_average_pr_interval'] = np.mean(pr_intervals)
            features[f'lead{j}_average_qrs_durations'] = np.mean(qrs_durations)
            features[f'lead{j}_average_qt_interval'] = np.mean(qt_intervals)
            ######################################## Frequency-Domain Features ########################################
            lf_power, hf_power, vlf_power, spectral_centroid, spectral_bandwidth, spectral_entropy, energy, std = frequency_domain_features(filtered)
            features[f'lead{j}_lf_power'] = lf_power
            features[f'lead{j}_hf_power'] = hf_power
            features[f'lead{j}_vlf_power'] = vlf_power
            features[f'lead{j}_spectral_centroid'] = spectral_centroid
            features[f'lead{j}_spectral_bandwidth'] = spectral_bandwidth
            features[f'lead{j}_spectral_entropy'] = spectral_entropy
            features[f'lead{j}_average_energy'] = np.mean(energy)
            features[f'lead{j}_average_std'] = np.mean(std)
        features_list.append(features)        
    return pd.DataFrame(features_list)

In [ ]:
train = process_features(train_hdf)

In [ ]:
test = process_features(test_hdf)

In [ ]:
train.to_csv('./input/v2processed_train.csv', index=False)

In [ ]:
test.to_csv('./input/v2processed_test.csv', index=False)